In [1]:
# this notebook shows how to obtain the basic results from Hunter et al. (2021)
# (corresponding to figs 3,4 & 5 / supp tables 2, 3, 6, 7, 8 & 9)
# it is reimplemented from the original paper:
# it uses a newer version of the EM model fitting code with a different M step
# (& a newer version of julia etc)
# and substitutes Julia equivalents for Matlab analyses
#
# so it does not precisely match results numerically but reproduces the results

In [2]:
# load some required packages
# ( you need to install these eg via import Pkg; Pkg.add(...) )

using CSV
using DataFrames
using LinearAlgebra
using SpecialFunctions:erf
using StatsFuns:logsumexp
using GLM
using Statistics
using StatsBase
using MixedModels
using CategoricalArrays

In [3]:
# load the EM model fitting code 
# get this from https://github.com/ndawlab/em

directory = "/home/daw/winhome/Dropbox/expts/julia em/git/em"

push!(LOAD_PATH,directory)
using EM

In [4]:
# patch in the python optimization function to produce results more similar to the original code
# (note you have to run this with only one thread / "unset JULIA_NUM_THREADS" ignoring warning)
# you can also skip this, & run julia multithreaded with the native optimizer to get (slightly
# different) results faster ;-)

using PyCall
using ForwardDiff

so = pyimport("scipy.optimize")

function EM.optimizesubject(likfun, startx)
    # call python's optimization function which used to work a little better than julia's
	# but is not thread safe
	a = so.minimize(likfun, startx, method="L-BFGS-B", jac = (x->ForwardDiff.gradient(likfun,x)))
	#println(a["message"])

	return((a["fun"],a["x"])::Tuple{Float64,Array{Float64,1}})
end

In [5]:
# load the model code (likelihood function)

include("patent likfuns.jl")

#3 (generic function with 1 method)

In [6]:
# load the data 

alldata = CSV.File("patent_investments_743subs.csv") |> DataFrame
exp1subs = unique(alldata.sub)[1:412]
exp1NS = length(exp1subs)

exp2subs = unique(alldata.sub)[413:end]
exp2NS = length(exp2subs)

allsubs = unique(alldata.sub)
allNS = length(allsubs)

743

In [7]:
# load the covariates
# make sure the covariates are z scored separately within cohorts

mergedcovars = CSV.File("mergedcovars.csv") |> DataFrame 
exp1covars = mergedcovars[1:412,:]
exp1covars.iqZ = zscore(exp1covars.iqZ)
exp1covars.lsasZ = zscore(exp1covars.lsasZ)

exp2covars = mergedcovars[413:end,:]
exp2covars.iqZ = zscore(exp2covars.iqZ)
exp2covars.lsasZ = zscore(exp2covars.lsasZ);

In [8]:
# Expt 1, basic model
# set up design matrix for fit

X = ones(exp1NS)

# start points

betas = [-1. 0 0 0 0 0 0 0 ]
sigma = [10.,1,1,1,1,1,1,1]

# fit the model

(betas,sigma,x,l,h) = em(alldata,exp1subs,X,betas,sigma,ewalik; emtol=1e-3, full=false, maxiter=1000);


iter: 667
betas: [1.23 1.2 -0.95 0.01 -0.34 -0.11 0.08 0.13]
sigma: [0.23, 0.61, 0.8, 0.01, 0.03, 0.02, 0.01, 0.01]
free energy: -31978.482325
change: [3.0e-6, 1.0e-6, -4.0e-6, 6.1e-5, -3.0e-6, -1.2e-5, 2.5e-5, 1.6e-5, 1.2e-5, 2.0e-6, 1.0e-6, 0.001, 0.000682, 4.7e-5, 0.000547, 0.000658]
max: 0.001


In [9]:
# x gives the per subject parameters (some need transforming to 0-1)
# extract them and regress on the covariates (originally we saved these and did regressions in Matlab)
# note significant effect of LSAS on w

exp1covars.beta = x[:,1]
exp1covars.alpha = 1 .- ( 0.5 .+ 0.5 .* erf.(x[:,2] ./ sqrt(2))) # alpha = 1-phi, squashed
exp1covars.w = ( 0.5 .+ 0.5 .* erf.(x[:,3] ./ sqrt(2)))

display(lm(@formula(beta~lsasZ+iqZ),exp1covars))
display(lm(@formula(alpha~lsasZ+iqZ),exp1covars))
display(lm(@formula(w~lsasZ+iqZ),exp1covars))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

beta ~ 1 + lsasZ + iqZ

Coefficients:
──────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%    Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)   1.23171      0.021374  57.63    <1e-99   1.18969      1.27372
lsasZ        -0.045646     0.021404  -2.13    0.0336  -0.0877215   -0.00357045
iqZ           0.0366528    0.021404   1.71    0.0876  -0.00542279   0.0787283
──────────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

alpha ~ 1 + lsasZ + iqZ

Coefficients:
──────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)   0.164033    0.00842711  19.46    <1e-59   0.147467    0.180599
lsasZ        -0.00789078  0.00843893  -0.94    0.3503  -0.0244799   0.00869832
iqZ           0.0110433   0.00843893   1.31    0.1914  -0.00554582  0.0276324
──────────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

w ~ 1 + lsasZ + iqZ

Coefficients:
──────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────
(Intercept)  0.217108   0.00952002  22.81    <1e-74  0.198394    0.235822
lsasZ        0.0224051  0.00953338   2.35    0.0192  0.00366457  0.0411456
iqZ          0.0224284  0.00953338   2.35    0.0191  0.00368782  0.0411689
──────────────────────────────────────────────────────────────────────────

In [10]:
# Repeat the process with the valenced model, expt 1

# set up the design matrix

X = ones(exp1NS)

# start points 

betas = [-1. 0 0 0 0 0 0 0 0]
sigma = [10.,1,1,1,1,1,1,1,1]

# fit the model

(betas,sigma,x,l,h) = em(alldata,exp1subs,X,betas,sigma,ewalik_valenced; emtol=1e-3, full=false, maxiter=1000);


iter: 687
betas: [1.28 1.29 -0.64 -1.46 0.02 -0.34 -0.1 0.08 0.13]
sigma: [0.24, 0.65, 0.87, 1.28, 0.0, 0.03, 0.02, 0.01, 0.0]
free energy: -31737.069446
change: [3.0e-6, 5.0e-6, -8.0e-6, -5.0e-6, 6.8e-5, -5.0e-6, -1.6e-5, 3.1e-5, 1.9e-5, 3.4e-5, 9.9e-5, 3.5e-5, 1.4e-5, 0.000999, 0.000666, 0.00015, 0.000474, 0.000638]
max: 0.000999


In [11]:
# x gives the per subject parameters (some need transforming to 0-1)
# extract them and regress on the covariates (originally we saved these and did regressions in Matlab)
# note significant effect of LSAS on wplus, not wminus

exp1covars.betavalenced = x[:,1]
exp1covars.alphavalenced = 1 .- ( 0.5 .+ 0.5 .* erf.(x[:,2] ./ sqrt(2))) # alpha = 1-phi, squashed
exp1covars.wplus = ( 0.5 .+ 0.5 .* erf.(x[:,3] ./ sqrt(2)))
exp1covars.wminus = ( 0.5 .+ 0.5 .* erf.(x[:,4] ./ sqrt(2)))

display(lm(@formula(betavalenced~lsasZ+iqZ),exp1covars))
display(lm(@formula(alphavalenced~lsasZ+iqZ),exp1covars))
display(lm(@formula(wplus~lsasZ+iqZ),exp1covars)) 
display(lm(@formula(wminus~lsasZ+iqZ),exp1covars))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

betavalenced ~ 1 + lsasZ + iqZ

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)   1.27565     0.0214658  59.43    <1e-99   1.23345     1.31784
lsasZ        -0.0395683   0.0214959  -1.84    0.0664  -0.0818245   0.00268797
iqZ           0.0341687   0.0214959   1.59    0.1127  -0.00808758  0.0764249
─────────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

alphavalenced ~ 1 + lsasZ + iqZ

Coefficients:
──────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)   0.150189    0.0082747   18.15    <1e-53   0.133923    0.166456
lsasZ        -0.00716423  0.00828631  -0.86    0.3878  -0.0234533   0.00912483
iqZ           0.00970181  0.00828631   1.17    0.2424  -0.00658726  0.0259909
──────────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

wplus ~ 1 + lsasZ + iqZ

Coefficients:
──────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────
(Intercept)  0.305973    0.0112457  27.21    <1e-93  0.283866    0.328079
lsasZ        0.0268589   0.0112615   2.39    0.0175  0.00472131  0.0489964
iqZ          0.0359981   0.0112615   3.20    0.0015  0.0138606   0.0581357
──────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

wminus ~ 1 + lsasZ + iqZ

Coefficients:
────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)   0.115345    0.00855081  13.49    <1e-33   0.0985359  0.132154
lsasZ         0.00607647  0.00856281   0.71    0.4783  -0.0107561  0.0229091
iqZ          -0.00128756  0.00856281  -0.15    0.8805  -0.0181202  0.0155451
────────────────────────────────────────────────────────────────────────────

In [12]:
# supp table 7: compare w+ to w- (interaction with lsas is only trending in Exp 1)
# using mixed model to capture repeated measures by subject
# note that Julia's MixedModels, unlike Matlab, doesnt use Satterthwaite DOF
# but this difference is minimal for this many subjects

exp1ws = DataFrame(w = [exp1covars.wplus;exp1covars.wminus], 
    sub = categorical([exp1subs;exp1subs]), 
    iqZ = [exp1covars.iqZ; exp1covars.iqZ], 
    lsasZ = [exp1covars.lsasZ; exp1covars.lsasZ], 
    valence = [ones(size(exp1subs));zeros(size(exp1subs))])

mm = (fit(MixedModel, @formula(w ~ 1 + valence * (lsasZ + iqZ) + (1 + valence * (lsasZ + iqZ) |sub)), exp1ws))

mm

|                 |    Est. |     SE |     z |      p |  σ_sub |
|:--------------- | -------:| ------:| -----:| ------:| ------:|
| (Intercept)     |  0.1149 | 0.0085 | 13.52 | <1e-40 | 0.1637 |
| valence         |  0.1911 | 0.0140 | 13.60 | <1e-41 | 0.2773 |
| lsasZ           |  0.0054 | 0.0088 |  0.62 | 0.5385 | 0.0378 |
| iqZ             | -0.0010 | 0.0081 | -0.13 | 0.8994 | 0.0111 |
| valence & lsasZ |  0.0238 | 0.0143 |  1.67 | 0.0957 | 0.0350 |
| valence & iqZ   |  0.0372 | 0.0135 |  2.76 | 0.0057 | 0.0144 |
| Residual        |  0.0393 |        |       |        |        |


In [13]:
# Expt 2, basic model
# set up design matrix for fit

X = ones(exp2NS)

# initial conditions

betas = [-1. 0 0 0 0 0 0 0 ]
sigma = [10.,1,1,1,1,1,1,1]

# fit the model

(betas,sigma,x,l,h) = em(alldata,exp2subs,X,betas,sigma,ewalik; emtol=1e-3, full=false, maxiter=1000);


iter: 910
betas: [1.13 1.24 -1.08 -0.05 -0.26 -0.17 0.09 0.15]
sigma: [0.43, 0.68, 1.26, 0.01, 0.01, 0.0, 0.02, 0.01]
free energy: -27648.934518
change: [1.0e-6, 0.0, -1.0e-6, -0.000121, -2.4e-5, -4.1e-5, 6.7e-5, 3.7e-5, 3.0e-6, 1.0e-6, 1.0e-6, 0.000174, 0.000999, 0.00096, 7.8e-5, 0.000398]
max: 0.000999


In [14]:
# x gives the per subject parameters (some need transforming to 0-1)
# extract them and regresss on the covariates (originally we saved these and did regressions in Matlab)
# note significant effect of LSAS on w

exp2covars.beta = x[:,1]
exp2covars.alpha = 1 .- ( 0.5 .+ 0.5 .* erf.(x[:,2] ./ sqrt(2))) # alpha = 1-phi, squashed
exp2covars.w = ( 0.5 .+ 0.5 .* erf.(x[:,3] ./ sqrt(2)))

display(lm(@formula(beta~lsasZ+iqZ),exp2covars))
display(lm(@formula(alpha~lsasZ+iqZ),exp2covars))
display(lm(@formula(w~lsasZ+iqZ),exp2covars))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

beta ~ 1 + lsasZ + iqZ

Coefficients:
────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)   1.13069      0.0333753  33.88    <1e-99   1.06503    1.19634
lsasZ        -0.00173645   0.033437   -0.05    0.9586  -0.0675145  0.0640416
iqZ           0.115738     0.033437    3.46    0.0006   0.0499601  0.181516
────────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

alpha ~ 1 + lsasZ + iqZ

Coefficients:
───────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error      t  Pr(>|t|)    Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)  0.161999   0.00943595  17.17    <1e-46   0.143436    0.180561
lsasZ        0.0156189  0.0094534    1.65    0.0995  -0.00297803  0.0342159
iqZ          0.010175   0.0094534    1.08    0.2826  -0.00842193  0.028772
───────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

w ~ 1 + lsasZ + iqZ

Coefficients:
────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)  0.204007     0.0130319  15.65    <1e-40   0.17837     0.229643
lsasZ        0.033322     0.013056    2.55    0.0112   0.00763802  0.059006
iqZ          0.00364774   0.013056    0.28    0.7801  -0.0220363   0.0293317
────────────────────────────────────────────────────────────────────────────

In [15]:
# Repeat the process with the valenced model, expt 2

# set up the design matrix

X = ones(exp2NS)

betas = [-1. 0 0 0 0 0 0 0 0]
sigma = [10.,1,1,1,1,1,1,1,1]

(betas,sigma,x,l,h) = em(alldata,exp2subs,X,betas,sigma,ewalik_valenced; emtol=1e-3, full=false, maxiter=1000);


iter: 722
betas: [1.19 1.29 -0.87 -1.25 -0.04 -0.27 -0.17 0.11 0.16]
sigma: [0.34, 0.68, 1.11, 1.27, 0.01, 0.01, 0.01, 0.01, 0.01]
free energy: -27433.19149
change: [3.0e-6, 1.0e-6, -3.0e-6, -6.0e-6, -6.8e-5, -1.7e-5, -2.6e-5, 2.3e-5, 1.4e-5, 1.0e-5, 2.0e-6, 1.0e-6, 3.0e-6, 0.000545, 0.001, 0.000912, 0.000317, 0.000446]
max: 0.001


In [16]:
# x gives the per subject parameters (some need transforming to 0-1)
# extract them and regresss on the covariates (originally we saved these and did regressions in Matlab)
# note significant effect of LSAS on wplus not wminus

exp2covars.betavalenced = x[:,1]
exp2covars.alphavalenced = 1 .- ( 0.5 .+ 0.5 .* erf.(x[:,2] ./ sqrt(2))) # alpha = 1-phi, squashed
exp2covars.wplus = ( 0.5 .+ 0.5 .* erf.(x[:,3] ./ sqrt(2)))
exp2covars.wminus = ( 0.5 .+ 0.5 .* erf.(x[:,4] ./ sqrt(2)))

display(lm(@formula(betavalenced~lsasZ+iqZ),exp2covars))
display(lm(@formula(alphavalenced~lsasZ+iqZ),exp2covars))
display(lm(@formula(wplus~lsasZ+iqZ),exp2covars))
display(lm(@formula(wminus~lsasZ+iqZ),exp2covars))

# also regress on additional psychiatric symptom factors.
# f3 is the one measuring social anxiety (significant against wplus not wminus)
# f1 & f2 control for other symptom dimensions (not signif)

display(lm(@formula(betavalenced~f1+f2+f3+iqZ),exp2covars))
display(lm(@formula(alphavalenced~f1+f2+f3+iqZ),exp2covars))
display(lm(@formula(wplus~f1+f2+f3+iqZ),exp2covars))
display(lm(@formula(wminus~f1+f2+f3+iqZ),exp2covars))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

betavalenced ~ 1 + lsasZ + iqZ

Coefficients:
───────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)   1.19397     0.0293161  40.73    <1e-99   1.1363     1.25164
lsasZ        -0.0149815   0.0293703  -0.51    0.6103  -0.0727595  0.0427964
iqZ           0.117881    0.0293703   4.01    <1e-04   0.0601027  0.175659
───────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

alphavalenced ~ 1 + lsasZ + iqZ

Coefficients:
────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)  0.152579    0.0093286   16.36    <1e-43   0.134228    0.17093
lsasZ        0.0128441   0.00934585   1.37    0.1703  -0.00554129  0.0312295
iqZ          0.00776495  0.00934585   0.83    0.4067  -0.0106204   0.0261503
────────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

wplus ~ 1 + lsasZ + iqZ

Coefficients:
──────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────
(Intercept)  0.255576    0.0134084  19.06    <1e-54   0.229199   0.281953
lsasZ        0.0462295   0.0134332   3.44    0.0007   0.0198035  0.0726555
iqZ          0.0082411   0.0134332   0.61    0.5400  -0.0181849  0.0346671
──────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

wminus ~ 1 + lsasZ + iqZ

Coefficients:
────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)   0.156699     0.0112417  13.94    <1e-34   0.134584   0.178814
lsasZ         0.00740131   0.0112625   0.66    0.5115  -0.0147546  0.0295572
iqZ          -0.0180707    0.0112625  -1.60    0.1096  -0.0402266  0.0040852
────────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

betavalenced ~ 1 + f1 + f2 + f3 + iqZ

Coefficients:
───────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)   1.19397     0.0293431  40.69    <1e-99   1.13624    1.2517
f1           -0.0261703   0.0325765  -0.80    0.4224  -0.0902571  0.0379165
f2            0.0200568   0.0298137   0.67    0.5016  -0.0385948  0.0787084
f3           -0.0160888   0.0327853  -0.49    0.6239  -0.0805862  0.0484087
iqZ           0.117773    0.0293975   4.01    <1e-04   0.0599404  0.175606
───────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

alphavalenced ~ 1 + f1 + f2 + f3 + iqZ

Coefficients:
────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)  0.152579    0.0093271   16.36    <1e-43   0.13423     0.170928
f1           0.0108461   0.0103549    1.05    0.2957  -0.00952475  0.0312169
f2           0.00168876  0.00947669   0.18    0.8587  -0.0169544   0.0203319
f3           0.00762334  0.0104212    0.73    0.4650  -0.012878    0.0281247
iqZ          0.00808648  0.00934437   0.87    0.3875  -0.0102964   0.0264694
────────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

wplus ~ 1 + f1 + f2 + f3 + iqZ

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                    Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)   0.255576      0.0134327  19.03    <1e-54   0.22915    0.282002
f1           -0.00698578    0.0149129  -0.47    0.6398  -0.0363235  0.0223519
f2           -0.000323906   0.0136482  -0.02    0.9811  -0.0271735  0.0265257
f3            0.0485017     0.0150085   3.23    0.0014   0.018976   0.0780274
iqZ           0.00860215    0.0134576   0.64    0.5231  -0.0178725  0.0350769
─────────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

wminus ~ 1 + f1 + f2 + f3 + iqZ

Coefficients:
──────────────────────────────────────────────────────────────────────────────
                    Coef.  Std. Error      t  Pr(>|t|)   Lower 95%   Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)   0.156699      0.0112645  13.91    <1e-34   0.134539   0.178859
f1            0.000429567   0.0125058   0.03    0.9726  -0.0241726  0.0250317
f2            0.00593095    0.0114451   0.52    0.6047  -0.0165847  0.0284466
f3            0.00749647    0.0125859   0.60    0.5518  -0.0172633  0.0322563
iqZ          -0.0179236     0.0112853  -1.59    0.1132  -0.0401249  0.00427765
──────────────────────────────────────────────────────────────────────────────

In [17]:
# supp table 7 & 9: compare w+ to w- (interaction with lsas / f3 is signif in Exp 2)
# using mixed model to capture repeated measures by subject
# note that Julia's MixedModels, unlike Matlab, doesnt use Satterthwaite DOF
# but this difference is minimal for this many subjects

exp2ws = DataFrame(w = [exp2covars.wplus;exp2covars.wminus], 
    sub = categorical([exp2subs;exp2subs]), 
    iqZ = [exp2covars.iqZ; exp2covars.iqZ], 
    lsasZ = [exp2covars.lsasZ; exp2covars.lsasZ], 
    f1 = [exp2covars.f1; exp2covars.f1], 
    f2 = [exp2covars.f2; exp2covars.f2], 
    f3 = [exp2covars.f3; exp2covars.f3], 
    valence = [ones(size(exp2subs));zeros(size(exp2subs))]
)

# supp tabel 7

mm = (fit(MixedModel, @formula(w ~ 1 + valence * (lsasZ + iqZ) + (1 + valence * (lsasZ + iqZ) |sub)), exp2ws))
display(mm)

# supp table 9

mm2 = (fit(MixedModel, @formula(w ~ 1 + valence * (f1 + f2 + f3 + iqZ) + (1 + valence * (f1 + f2 + f3 + iqZ) |sub)), exp2ws))
display(mm2)

|                 |    Est. |     SE |     z |      p |  σ_sub |
|:--------------- | -------:| ------:| -----:| ------:| ------:|
| (Intercept)     |  0.1569 | 0.0112 | 14.02 | <1e-43 | 0.1612 |
| valence         |  0.0963 | 0.0167 |  5.78 | <1e-08 | 0.2299 |
| lsasZ           |  0.0073 | 0.0114 |  0.65 | 0.5187 | 0.0268 |
| iqZ             | -0.0181 | 0.0120 | -1.51 | 0.1310 | 0.0316 |
| valence & lsasZ |  0.0410 | 0.0169 |  2.43 | 0.0151 | 0.0293 |
| valence & iqZ   |  0.0324 | 0.0190 |  1.71 | 0.0877 | 0.1139 |
| Residual        |  0.1178 |        |       |        |        |


|               |    Est. |     SE |     z |      p |  σ_sub |
|:------------- | -------:| ------:| -----:| ------:| ------:|
| (Intercept)   |  0.1304 | 0.0091 | 14.36 | <1e-46 | 0.1091 |
| valence       |  0.1058 | 0.0160 |  6.60 | <1e-10 | 0.2542 |
| f1            | -0.0123 | 0.0126 | -0.97 | 0.3296 | 0.1341 |
| f2            | -0.0058 | 0.0095 | -0.61 | 0.5440 | 0.1035 |
| f3            |  0.0245 | 0.0124 |  1.97 | 0.0485 | 0.0922 |
| iqZ           | -0.0022 | 0.0092 | -0.25 | 0.8060 | 0.0942 |
| valence & f1  | -0.0119 | 0.0176 | -0.68 | 0.4992 | 0.0700 |
| valence & f2  | -0.0039 | 0.0144 | -0.27 | 0.7855 | 0.0670 |
| valence & f3  |  0.0464 | 0.0180 |  2.57 | 0.0101 | 0.0518 |
| valence & iqZ |  0.0313 | 0.0189 |  1.65 | 0.0987 | 0.1564 |
| Residual      |  0.0017 |        |       |        |        |
